# PD881
## settings
## raw data

In [1]:
import pandas as pd

# 文件位置定义
file_col_def = './data/PD881_Column_Definition_20201014.xlsx'
# file_raw_data = './data/PD881_raw.xlsx'
# file_data = './data/PD881_data.xlsx'
file_raw_data = './data/PD876_raw.xlsx'
file_data = './data/PD876_data.xlsx'
file_final_data = './data/PD876_final.xlsx'

COL_IDX = "NIDonly"
LABEL_IDX = "CATEGORY"

In [8]:
# 列名信息
df_col_def = pd.read_excel(file_col_def)
# col_def = df_col_def[df_col_def['SPSS数据保留']==1]['英文描述'].to_list()
col_def = df_col_def[df_col_def['数据分析保留']==1]['英文描述'].to_list()
# col_def

In [9]:
# 原数据初过滤列，生成初始数据
df_raw_data = pd.read_excel(file_raw_data, index_col=COL_IDX)
df = df_raw_data[col_def]
df.to_excel(file_data)
del df_raw_data

## preprocessing
### load file

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler


file_col_def = './data/PD881_Column_Definition_20201014.xlsx'
# file_data = './data/PD881_data.xlsx'
file_data = './data/PD876_data.xlsx'
file_final_data = './data/PD876_final.xlsx'
COL_IDX = "NIDonly"
LABEL_IDX = "CATEGORY"

# load init data
print("0. Load file", file_data)
df = pd.read_excel(file_data, index_col=COL_IDX)

0. Load file ./data/PD876_data.xlsx


### EDA

In [ ]:
# 探索性分析
# df.describe(include="all").to_excel('./temp/df_describe_876.xlsx')

In [12]:
# df["CATEGORY"].unique()

array(['阴性', 'LRRK2 2385', 'PLA2G6+LRRK2 2385', 'PARK2',
       'PARK2复合杂合？（de novo无法判定)', 'GBA', '阴性（归入晚发散发）', 'DJ-1',
       'PARK2+PINK1 双基因，复合杂合', 'PARK2未验证', 'PLA2G6', 'LRRK2 2385（归入晚发散发）',
       'SNCA', 'LRRK2', '阴性（近亲结婚，父亲母亲去世，有3位患者去世一位）', '阴性（CHCHD2 VUS）',
       'DJ-1+LRRK2 2385', 'GBA+LRRK2 2385', 'GBA(de novo)', 0, 'PARK2单杂合',
       'LRRK2 2385纯和', 'GBA需要验证排除复合杂合，排除嵌合体', '阴性（康旭做的，有机会可以重做）',
       'PARK2+LRRK2 2385', 'PARK2 未验证', '阴性（大家系）', 'PARK2单杂合+LRRK2 2385',
       'CHCHD2', nan, 'PINK1', 'PINK1未验证', 'GBA（归入晚发散发）', 'PARK2未验证（双生子）',
       'VPS35', 'CHCHD2+LRRK2 2385', 'GBA（N227R，有2个碱基突变）',
       'PARK2+PSEN2(验证非常重要，详细问家族史)'], dtype=object)

### rename columns

In [ ]:
# df = df.rename(columns={
#     'ori_col_name': 'new_col_name',
# })

### special rules

In [2]:
# column familyhis
def familyhis_replace_rule(x):
    if x.values[0] == ",":
        return np.nan
    elif x.values[0] == ",无,":
        return "无"
    else:
        return "是"

df[["familyhis"]] = df[["familyhis"]].apply(familyhis_replace_rule, axis=1)

# df.loc[:, "familyhis"] = df[["familyhis"]].apply(familyhis_replace_rule, axis=1)[0]

In [ ]:
# column V0_DIAG_HY
# 五分类 -> 十分类, 不需要了
# df[["V0_DIAG_HY"]] = df[["V0_DIAG_HY"]].apply(lambda x: 2 * x)

### drop rows

In [3]:
# 删除以下数据样本，其中单个样本超过40%的列为空值
len_cols = len(df.columns)
null_value_threshold = int(len_cols * 0.4)

drop_row_list = df[df.isnull().sum(axis=1) >= null_value_threshold].index.to_list()

print("1. Drop values")
print("1-1. Drop", len(drop_row_list), "rows, export to ./temp/rows_dropped_876.xlsx")
pd.DataFrame(data=drop_row_list, columns=[COL_IDX]).to_excel("./temp/rows_dropped_876.xlsx")
df.drop(drop_row_list, axis=0, inplace=True)

1. Drop values
1-1. Drop 317 rows, export to ./temp/rows_dropped_876.xlsx


### drop columns

In [4]:
# 删除以下数据特征，其中所有样本该数据特征超过40%为空值
len_rows = len(df)
null_col_threshold = int(len_rows * 0.4)
# drop_col_list = df.columns[df.describe(include="all").loc["count", :] <= null_col_threshold].to_list()

drop_col_list = df.columns[df.isnull().sum(axis=0) >= null_col_threshold].to_list()

print("1-2. Drop", len(drop_col_list), "cols, export to ./temp/cols_dropped_876.xlsx")
pd.DataFrame(data=drop_col_list, columns=["DROPPED_COLUMNS"]).to_excel("./temp/cols_dropped_876.xlsx")
df.drop(drop_col_list, axis=1, inplace=True)

1-2. Drop 4 cols, export to ./temp/cols_dropped_876.xlsx


In [ ]:
df.describe()

### encoding

In [32]:
# 定类列名信息（离散）
df_col_def = pd.read_excel(file_col_def)
col_discrete_def = \
    df_col_def[(df_col_def["数据分析保留"]==1)&(df_col_def["数据类型"]=="定类")]["英文描述"].to_list()
# col_discrete_def

# pandas onehot编码
df = pd.get_dummies(df, columns=col_discrete_def)

In [ ]:
# 离散数据序数编码器（如使用onehot，不要执行ordinal编码）
ordinal_enc_dict = {}
for col_name in col_discrete_def:
    # Create Ordinal encoder for col
    ordinal_enc_dict[col_name] = OrdinalEncoder(dtype="int64")
    col = df[col_name]
    
    # Select non-null values of col
    col_not_null = col[col.notnull()]
    reshaped_vals = col_not_null.values.reshape(-1, 1)
    encoded_vals = ordinal_enc_dict[col_name].fit_transform(reshaped_vals)
    
    # Store the values to non-null values of the column in users
    df.loc[col.notnull(), col_name] = np.squeeze(encoded_vals)

# 离散数据编码映射
{key: val.categories_[0].tolist() for key, val in ordinal_enc_dict.items()}

# for val in ordinal_enc_dict.values():
#     print(val.categories_)

### imputing

In [33]:
# 均值补差
imputer = SimpleImputer(strategy='mean')
df[:] = imputer.fit_transform(df)

### scaling

In [ ]:
# min-max normalization
min_max_scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
df[:] = min_max_scaler.fit_transform(df)

In [34]:
# z-score normalization
standard_scaler = StandardScaler(copy=True)
df[:] = standard_scaler.fit_transform(df)

### check missing values

In [35]:
# check missing values
print("2. Check missing values")
print("2-1. Index - Missing values detected:", df.index.isna().any())

col_missing_values = df.columns[df.isna().any().values==True].tolist()
print("2-2. Columns - Missing values detected:", len(col_missing_values)>0)
# print(col_missing_values)

2. Check missing values
2-1. Index - Missing values detected: False
2-2. Columns - Missing values detected: False


### export final data

In [36]:
file_final_data = './data/PD876_final.xlsx'
df.to_excel(file_final_data)

## Data Process

In [2]:
import numpy as np
import pandas as pd


file_col_def = './data/PD881_Column_Definition_20201014.xlsx'
file_final_data = './data/PD876_final.xlsx'
COL_IDX = "NIDonly"

df = pd.read_excel(file_final_data, index_col=COL_IDX)

In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.9)
# pca2 = PCA(n_components=0.3)
# pca3 = PCA(n_components="mle")
matrix_pca = pca.fit_transform(df)
matrix_pca
# pca.explained_variance_ratio_
m = pca.inverse_transform(matrix_pca)
# variance_pca = pca.explained_variance_ratio_.sum()
# variance_pca
m

array([[ 0.59342621,  0.59277928,  0.33134117, ..., -0.04203872,
         1.02423405, -0.02447817],
       [ 0.35632879,  0.40952064,  0.58979016, ..., -0.01351738,
         1.06074802, -0.004593  ],
       [ 0.61885123,  0.70958841,  0.57502815, ..., -0.02384252,
         0.78810213,  0.06287219],
       ...,
       [ 0.6389589 ,  0.68574599,  0.65821021, ..., -0.0318617 ,
         1.35482777, -0.18844095],
       [ 0.64290454,  0.74865348,  0.63652534, ...,  0.05657575,
         0.94022751,  0.06584416],
       [ 0.42649832,  0.57282219,  0.66945106, ..., -0.08976786,
         1.10140162, -0.19153202]])

In [24]:
df.describe(include='all')

,INPUT_AGE,ageonset,HEIGHT,FIRST_DIAG_AGE,PMH_ANESTHETIC,V0_AGE,V0_COURSE_MONTH,V0_DIAG_HY,V0_PD_WEIGHT,V0_BP_SIT_SYS,...,V0_SLEEP_REMIT_无,V0_SLEEP_REMIT_是,V0_dementia_无,V0_dementia_是,V0_dyskinesia_无,V0_dyskinesia_是,V0_ILLUSION_无,V0_ILLUSION_是,V0_PKSMEDICINE_无,V0_PKSMEDICINE_是
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,...,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000
mean,0.507590,0.577516,0.475796,0.553915,0.054143,0.496790,0.177054,0.283498,0.404471,0.096446,...,0.416816,0.568873,0.794275,0.119857,0.840787,0.153846,0.908766,0.082290,0.921288,0.035778
std,0.182631,0.174223,0.166987,0.165006,0.110541,0.187400,0.173470,0.216446,0.165654,0.056635,...,0.493473,0.495677,0.404592,0.325085,0.366202,0.361124,0.288200,0.275052,0.269530,0.185903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.378571,0.467532,0.333333,0.457831,0.000000,0.367647,0.057746,0.250000,0.280576,0.086429,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,0.500000,0.571429,0.488889,0.554217,0.000000,0.485294,0.115493,0.250000,0.404471,0.096446,...,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,0.671429,0.701299,0.600000,0.662651,0.125000,0.661765,0.228169,0.500000,0.496403,0.096446,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
